In [27]:
%pip install requests

%pip install beautifulsoup4

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.0-cp311-cp311-win_amd64.whl.metadata (34 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.0-cp311-cp311-win_amd64.whl (101 kB)
Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)
Note: you may need to restart the kernel to use updated packages.

  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)


In [26]:
%pip install pandas

  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl (12.9 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

headers = {
"accept-ranges": "bytes",
"cache-control": "no-store,must-revalidate",
"content-type": "application/json;charset=utf-8",
"date": "Fri, 06 Dec 2024 18:04:11 GMT",
"server":"envoy",
"vary":"Accept-Encoding",
"via":"1.1 varnish",
"x-cache":"MISS",
"x-cache-hits":"0",
"x-content-encoding-over-network":"br"
}

# URL of the TripAdvisor page
url = "https://www.tripadvisor.fr/"

# Send a GET request
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.post(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

print(soup.prettify())


<html>
 <head>
  <title>
   tripadvisor.fr
  </title>
  <style>
   #cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}
  </style>
 </head>
 <body style="margin:0">
  <p id="cmsg">
   Please enable JS and disable any ad blocker
  </p>
  <script data-cfasync="false">
   var dd={'rt':'c','cid':'AHrlqAAAAAMAUYSyGzeLpBIAJUEvBA==','hsh':'2F05D671381DB06BEE4CC52C7A6FD3','t':'fe','r':'b','s':46694,'e':'ed3e7e4675400fe9723a50eb448b93e7656477f06e55007a448165c26a3daef7','host':'geo.captcha-delivery.com','cookie':'rsmHpsLpOSXcbJQVr1Xu9mnz57uvzt1pRQMHIt5IvDoA3aHm3ZCz3auOtv01gyjVhod2O3537K6eA3Wa4OC4xPjI8jYG4XfvMCgRLyl6X2AGXldVwPjtDrnCkCuJCsKg'}
  </script>
  <script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js">
  </script>
 </body>
</html>



In [37]:
%pip install requests-html
%pip install lxml[html_clean]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from requests_html import HTML

doc = "https://www.tripadvisor.fr/"
s = HTML(html=doc)

In [6]:
s.url

'https://example.org/'

In [24]:
print(soup.prettify())

<html>
 <head>
  <title>
   tripadvisor.fr
  </title>
  <style>
   #cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}
  </style>
 </head>
 <body style="margin:0">
  <p id="cmsg">
   Please enable JS and disable any ad blocker
  </p>
  <script data-cfasync="false">
   var dd={'rt':'c','cid':'AHrlqAAAAAMAj2ooc_BchhEAJUEvBA==','hsh':'2F05D671381DB06BEE4CC52C7A6FD3','t':'fe','r':'b','s':46694,'e':'d2f9639e0430e9ae6034371ed42a5bb6da97214cd32770405835f4aaacfdb533','host':'geo.captcha-delivery.com','cookie':'ZemQ~Xr6RJnqfYpFvht8O873WuB4Jrd3ozhdgwCiHMkkPHvyF6Q5j4ehFaS5apU~ZAf9YKPYvwijMi~z2UGrJKJ~ms20AHdL3hlpCSlIczbjI99GjgnE~qVQhgIqI~6S'}
  </script>
  <script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js">
  </script>
 </body>
</html>

